In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp

In [17]:
demand=pd.read_csv("DemandTotal.csv")
demand.head()

,Unnamed: 0,demand,date,time,EVdemand,demandW/OEV,demandZEV,demand1/2ZEV
0,0,22443,20180101,0,206.62428,21306.56646,38663.00598,29984.78622
1,1,23613,20180101,1,163.23168,22715.22576,36426.68688,29570.95632
2,2,26544,20180101,2,131.13972,25822.73154,36838.46802,31330.59978
3,3,26696,20180101,3,106.42416,26110.66712,35050.29656,30580.48184
4,4,25926,20180101,4,93.13560,25413.75420,33237.14460,29325.44940


In [18]:
demandBase=demand['demand'].to_numpy()
demandZEV=demand['demandZEV'].to_numpy()
demandZEVHalf=demand['demand1/2ZEV'].to_numpy()
demandBase.shape

(8760,)

In [19]:
generator=pd.read_csv("GeneratorAggregated.csv")
generator.head()

,variableCost,Technology,Nameplate.Capacity..MW.,Minimum.Load..MW.
0,0.0,Batteries,233.6,27.3
1,0.0,Nuclear,2323.0,0.0
2,0.0,Solar Photovoltaic,10543.7,0.0
3,1.7,Conventional Hydroelectric,10068.4,780.6
4,2.2,Onshore Wind Turbine,6077.8,520.2


In [20]:
capacityMax=np.repeat(generator['Nameplate.Capacity..MW.'].to_numpy().reshape(1,40),8760).reshape(40,8760).T
capacityMax.shape
# np.sum(capacityMax, axis=1)


(8760, 40)

In [24]:
capacityMin=np.repeat(generator['Minimum.Load..MW.'].to_numpy().reshape(1,40),8760).reshape(40,8760).T
capacityMin.shape

# capacityMin<capacityMax

(8760, 40)

In [29]:
capacityZero=np.zeros((8760,40))
capacityZero.shape



(8760, 40)

In [26]:
cost=generator['Nameplate.Capacity..MW.'].to_numpy().reshape(40,1)
cost.shape

(40, 1)

In [10]:
# demandBase.shape
# demandBase

In [28]:
capacityMax

array([[  233.6,  2323. , 10543.7, ...,    92.7,    58.4,   223.5],
       [  233.6,  2323. , 10543.7, ...,    92.7,    58.4,   223.5],
       [  233.6,  2323. , 10543.7, ...,    92.7,    58.4,   223.5],
       ...,
       [  233.6,  2323. , 10543.7, ...,    92.7,    58.4,   223.5],
       [  233.6,  2323. , 10543.7, ...,    92.7,    58.4,   223.5],
       [  233.6,  2323. , 10543.7, ...,    92.7,    58.4,   223.5]])

In [12]:
# capacityMin.shape
# np.sum(capacityMax,axis=1)>=demandBase
# np.sum(capacityMin,axis=1)<=demandBase

In [13]:
# np.sum(np.sum(capacityMin,axis=1)*cost)
# np.sum(np.sum(capacityMax,axis=1)*cost)

In [30]:
# Create two scalar optimization variables.
r = cp.Variable((8760,40)) # a matrix
# cp.sum(r,axis=1).shape
# type(r)

# Create two constraints.
constraints = [ cp.sum(r, axis=1) == demandBase,
               r<=capacityMax,
               r>=capacityZero]
# Form objective.
obj = cp.Minimize(cp.sum(r@cost))
# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve() # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", r.value)

SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.

In [15]:
type(r)

cvxpy.expressions.variable.Variable